<a href="https://colab.research.google.com/github/lucabmanning/563-luca/blob/main/Luca_Project1_Revision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy
import matplotlib.pyplot as plt

def pbc(d,L):
  for i,x in enumerate(d):
    if x > L/2:
      d[i] = x-L
    elif x <= -L/2:
      d[i] = x+L
  return d

def distance(ri,rj,L):
  d = pbc(rj-ri,L)
  return numpy.linalg.norm(d)

def lj(r,epsilon=1.,sigma=1.):
  return 4*epsilon*((sigma/r)**12-(sigma/r)**6)

class simulation():
  def __init__(self,N=100,L=10,T=1.,maxD=0.1,maxSteps=10,debug=False,seed=2):
    numpy.random.seed(seed)
    self.N = N
    self.L = L
    self.T = T
    self.maxD = maxD
    self.accept = 0
    self.reject = 0
    self.rcut = 2.5
    self.pos = numpy.zeros((N,3))
    self.debug = debug
    self.maxSteps = maxSteps
    self.energies = numpy.zeros(maxSteps)
    for i in range(self.N):
      while True:
        self.pos[i] = numpy.random.uniform(low=-L/2,high=L/2,size=3)
        if self.energy(i,initializing=True)<=0.:
          break
    self.E = self.energy()
    for i in range(self.maxSteps):
      self.trial()
      if self.debug:
        print("{:03}:   {:.2f}".format(i, self.E))
      self.energies[i] = self.E

  def energy(self,i=None,initializing=False):
    E = 0
    if i is not None and initializing:
      pi = self.pos[i]
      for j in range(i):
        r = distance(pi,self.pos[j],self.L)
        if r <= self.rcut:
          E += lj(r)
      return E
    if i is None:
      for i in range(self.N-1):
        pi = self.pos[i]
        for j in range(i+1,self.N):
          r = distance(pi,self.pos[j],self.L) # speed up with numpy
          if r <= self.rcut:
            E += lj(r)
      return E

  def trial(self):
    i = numpy.random.randint(self.N)
    oldP = numpy.copy(self.pos[i])
    oldE = self.E
    oldEi = self.energy()
    d = numpy.random.uniform(low=-self.maxD,high=self.maxD,size=3)
    newP = pbc(oldP+d,self.L)
    self.pos[i] = newP
    newEi = self.energy()
    dE = newEi - oldEi
    if numpy.random.uniform() <= numpy.exp(-dE/self.T):
      if self.debug:
        print("accept")
      self.accept += 1
      self.E += dE
      return
    else:
      if self.debug:
        print("reject")
      self.pos[i] = oldP
      self.reject += 1
      self.E = oldE
      return

In [2]:
def autocorr1D(array):
  ft = numpy.fft.rfft(array - numpy.average(array))
  acorr = numpy.fft.irfft(ft * numpy.conjugate(ft)) / (len(array) * numpy.var(array))
  dt = numpy.where(acorr<0)[0][0]
  nsamples = len(array)//dt
  acorr = acorr[dt:nsamples]
  #return acorr[0 : len(acorr) // 2]
  return nsamples, dt


In [ ]:
density = 0.5
N = 32
V = N/density
L = V**(1/3)
print(L)
maxSteps = 10000
s = simulation(N=N, L=L, maxSteps = maxSteps,debug=False,seed=23)
print("{:02f} +/- {:02f}".format(s.energies.mean()/N, s.energies.std()/N))

3.9999999999999996


In [ ]:
plt.plot(s.energies[0:]/N)
plt.show()

In [ ]:
for start in range(0,10000,100):
  print(start, autocorr1D(s.energies[start:]))


In [ ]:
real_average = s.energies[4700::342].mean()/N
real_std = s.energies[4700::342].std()/N
print(real_average, real_std)

In [ ]:
ns,dt = autocorr1D(s.energies[4700:])
print(ns,dt)

In [ ]:
start = 5000
steps = 100
plt.plot(range(start,start+steps),s.energies[start:start+steps]/N)
plt.show()
print("N={}, AverageE: {:.2f}".format(N,s.energies[maxSteps//2:].mean()/N))
# per particle around -4.25 with  N=32, density .7, L=3.57, 10k maxSteps
# per particle around -3.91 with  N=20, density .7, L=3.057, 10k maxSteps
# per particle around  with  N=28, density .7, L=3.057, 10k maxSteps


In [ ]:
for N in range(20,51,5):
  density = 0.5
  V = N/density
  L = V**(1/3)
  maxSteps = 10000
  s = simulation(N=N, L=L, maxSteps = maxSteps,debug=False)
  print("N={}, AverageE: {:.2f}".format(N,s.energies[maxSteps//2:].mean()/N))

In [ ]:
import time
for N in range(20,51,5):
    density = 0.7
    V = N/density
    L = V**(1/3)
    maxSteps = 10000
    t1 = time.perf_counter()
    s = simulation(N=N, L=L, maxSteps = maxSteps,debug=False)
    t2 = time.perf_counter()
    print("\nN=",N,"; {:.2f} seconds/sim".format(t2-t1))
    print("N={}, AverageE: {:.2f}".format(N,s.energies[maxSteps//2:].mean()/N))